In [1]:
import os
import time
import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# Load API key from .env (env var name per your setup)
load_dotenv()
API_KEY = os.getenv("ALPHA_VANTAGE_KEY")
assert API_KEY, "Missing ALPHA_VANTAGE_KEY in .env"

BASE_URL = "https://www.alphavantage.co/query"

RAW = Path("../data/raw")
RAW.mkdir(parents=True, exist_ok=True)

TICKERS = ["AAPL", "MSFT", "SPY"]  # SPY as S&P 500 proxy

In [2]:
def _call_alpha_vantage(params, max_retries=3, sleep_seconds=15):
    """Low-level call with retry on rate-limit Note."""
    for attempt in range(1, max_retries + 1):
        r = requests.get(BASE_URL, params=params, timeout=30)
        r.raise_for_status()
        js = r.json()
        if "Note" in js:
            if attempt == max_retries:
                raise RuntimeError(f"Rate limit hit; last payload: {js}")
            time.sleep(sleep_seconds)
            continue
        return js
    raise RuntimeError("Unexpected loop exit in _call_alpha_vantage")


def pull_daily(symbol: str, outputsize: str = "full", try_adjusted_first: bool = True) -> pd.DataFrame:
    """
    Pull daily data for a symbol. Tries TIME_SERIES_DAILY_ADJUSTED first.
    Falls back to TIME_SERIES_DAILY if adjusted is not available.
    """
    if try_adjusted_first:
        params_adj = {
            "function": "TIME_SERIES_DAILY_ADJUSTED",
            "symbol": symbol,
            "outputsize": outputsize,
            "apikey": API_KEY,
            "datatype": "json",
        }
        js = _call_alpha_vantage(params_adj)
        ts = js.get("Time Series (Daily)")
        if ts:
            df = pd.DataFrame.from_dict(ts, orient="index", dtype="float")
            df.index = pd.to_datetime(df.index)
            df.sort_index(inplace=True)
            df["symbol"] = symbol
            df.attrs["schema"] = "adjusted"
            return df
        else:
            print(f"[{symbol}] Adjusted endpoint unavailable; fallback to DAILY")

    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "outputsize": outputsize,
        "apikey": API_KEY,
        "datatype": "json",
    }
    js = _call_alpha_vantage(params)
    ts = js.get("Time Series (Daily)")
    if not ts:
        raise ValueError(f"[{symbol}] DAILY endpoint unavailable. Payload: {js}")
    df = pd.DataFrame.from_dict(ts, orient="index", dtype="float")
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
    df["symbol"] = symbol
    df.attrs["schema"] = "daily"
    return df

In [3]:
def keep_recent_years(df: pd.DataFrame, years: int = 2) -> pd.DataFrame:
    """Return only the last `years` years of data."""
    cutoff = pd.Timestamp.today().normalize() - pd.DateOffset(years=years)
    return df.loc[df.index >= cutoff].copy()

In [4]:
frames_recent = {}
for i, t in enumerate(TICKERS, start=1):
    df_full = pull_daily(t, outputsize="full", try_adjusted_first=True)

    # Trim to last 2 years
    df_recent = keep_recent_years(df_full, years=2)
    frames_recent[t] = df_recent

    out_path = RAW / f"{t}_daily.csv"
    df_recent.to_csv(out_path)

    print(f"Saved {t}: {out_path}  rows={len(df_recent)}  (schema={df_full.attrs.get('schema')})")

    if i < len(TICKERS):  # respect API limits
        time.sleep(12)

[AAPL] Adjusted endpoint unavailable; fallback to DAILY
Saved AAPL: ../data/raw/AAPL_daily.csv  rows=501  (schema=daily)
[MSFT] Adjusted endpoint unavailable; fallback to DAILY
Saved MSFT: ../data/raw/MSFT_daily.csv  rows=501  (schema=daily)
[SPY] Adjusted endpoint unavailable; fallback to DAILY
Saved SPY: ../data/raw/SPY_daily.csv  rows=501  (schema=daily)


In [5]:
def validate_symbol(symbol: str) -> pd.DataFrame:
    """
    Validate the CSV in data/raw/.
    Works for both adjusted and daily schemas.
    """
    path = RAW / f"{symbol}_daily.csv"
    assert path.exists() and path.stat().st_size > 0, f"[{symbol}] Missing or empty file"
    df = pd.read_csv(path, parse_dates=[0], index_col=0)

    is_adjusted = "5. adjusted close" in df.columns
    required = ([
        "1. open", "2. high", "3. low", "4. close",
        "5. adjusted close", "6. volume", "7. dividend amount", "8. split coefficient",
        "symbol",
    ] if is_adjusted else [
        "1. open", "2. high", "3. low", "4. close",
        "5. volume", "symbol",
    ])

    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"[{symbol}] Missing columns: {missing}")

    if df.isna().any().any():
        raise ValueError(f"[{symbol}] Missing values detected.")

    print(f"{symbol} OK — rows={len(df)}, {df.index.min().date()} → {df.index.max().date()}")
    return df


validated = {t: validate_symbol(t) for t in TICKERS}

AAPL OK — rows=501, 2023-08-21 → 2025-08-19
MSFT OK — rows=501, 2023-08-21 → 2025-08-19
SPY OK — rows=501, 2023-08-21 → 2025-08-19
